In [1]:
from gurobipy import Model, GRB

# Sets
T = [1, 2, 3]   # Time periods
J = [1]         # Single VPP for test case

# Parameters
                 
lambda_CEP = {1: 0.4, 2: 0.6, 3: 1.4}   # Selling price to wholesale market
lambda_CES = {1: 0.2, 2: 0, 3: 0}   # Buying price from wholesale market
M = 100  # Big-M value

# Fixed VPP responses (from lower-level test case)
P_VPP_p = {
    (1, 1): 2.999999999999093,
    (1, 2): 2.999999999997612,
    (1, 3): 2.9999999999989884
}

P_VPP_s = {
    (1, 1): 0.0,
    (1, 2): 0.0,
    (1, 3): 0.0
}


# Initialize model
model = Model('DSO_Upper_Level_Test')

# Decision Variables
lambda_ES = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_ES_{t}") for t in T}
lambda_EP = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_EP_{t}") for t in T}

P_DSO = {t: model.addVar(name=f"P_DSO_{t}") for t in T}
P_DSO_p = {t: model.addVar(lb=0, name=f"P_DSO_p_{t}") for t in T}
P_DSO_s = {t: model.addVar(lb=0, name=f"P_DSO_s_{t}") for t in T}

z = {t: model.addVar(vtype=GRB.BINARY, name=f"z_{t}") for t in T}

# Constraints
for t in T:
    # Power balance
    model.addConstr(P_DSO[t] == sum(P_VPP_p[j, t] - P_VPP_s[j, t] for j in J), name=f"Power_Balance_{t}")

    # Big-M constraints for Purchase Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO[t], name=f"BM_Purch_LB_{t}")
    model.addConstr(P_DSO[t] <= M * z[t], name=f"BM_Purch_UB_{t}")
    model.addConstr(-M * (1 - z[t]) <= P_DSO_p[t] - P_DSO[t], name=f"BM_Purch_Link_{t}")
    model.addConstr(P_DSO_p[t] <= M * z[t], name=f"BM_Purch_PBound_{t}")

    # Big-M constraints for Sale Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO_s[t], name=f"BM_Sale_LB_{t}")
    model.addConstr(P_DSO_s[t] <= M * (1 - z[t]), name=f"BM_Sale_UB_{t}")
    model.addConstr(-M * z[t] <= P_DSO_s[t] + P_DSO[t], name=f"BM_Sale_Link_{t}")

# Objective Function
profit = sum(
    lambda_CES[t] * P_DSO_s[t] - lambda_CEP[t] * P_DSO_p[t] +
    lambda_EP[t] * sum(P_VPP_p[j, t] for j in J) -
    lambda_ES[t] * sum(P_VPP_s[j, t] for j in J)
    for t in T
)

model.setObjective(profit, GRB.MAXIMIZE)

# Optimize
model.optimize()

# Display Results
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")

print(f"Optimal Profit: {model.ObjVal}")


Set parameter Username
Set parameter LicenseID to value 2644336
Academic license - for non-commercial use only - expires 2026-03-31
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 18 columns and 51 nonzeros
Model fingerprint: 0x7fb4c1a9
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 3e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 24 rows and 18 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: -3.21165e-12 
No other solutions better than 6.57252e-14

Optimal solution found (

In [16]:
model.write("DSO_Upper_Level_Test.lp")  # Save the model to a file

In [2]:
from gurobipy import Model, GRB

# Time periods and VPPs
T = [1, 2, 3]
J = [1, 2, 3]  # Multiple VPPs

# Prices
lambda_CEP = {1: 0.4, 2: 0.6, 3: 1.4}  # Selling price to wholesale
lambda_CES = {1: 0.2, 2: 0, 3: 0}      # Buying price from wholesale
M = 100  # Big-M constant

# Dummy data from lower level solutions for 3 VPPs
P_VPP_p = {(j, t): 3.0 for j in J for t in T}
P_VPP_s = {(j, t): 0.0 for j in J for t in T}

# Create model
model = Model("DSO_Upper_Level_General")

# Variables
lambda_ES = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_ES_{t}") for t in T}
lambda_EP = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_EP_{t}") for t in T}
P_DSO = {t: model.addVar(name=f"P_DSO_{t}") for t in T}
P_DSO_p = {t: model.addVar(lb=0, name=f"P_DSO_p_{t}") for t in T}
P_DSO_s = {t: model.addVar(lb=0, name=f"P_DSO_s_{t}") for t in T}
z = {t: model.addVar(vtype=GRB.BINARY, name=f"z_{t}") for t in T}

# Constraints
for t in T:
    model.addConstr(P_DSO[t] == sum(P_VPP_p[j, t] - P_VPP_s[j, t] for j in J), name=f"Power_Balance_{t}")

    # Purchase Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO[t])
    model.addConstr(P_DSO[t] <= M * z[t])
    model.addConstr(-M * (1 - z[t]) <= P_DSO_p[t] - P_DSO[t])
    model.addConstr(P_DSO_p[t] <= M * z[t])

    # Sale Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO_s[t])
    model.addConstr(P_DSO_s[t] <= M * (1 - z[t]))
    model.addConstr(-M * z[t] <= P_DSO_s[t] + P_DSO[t])

# Objective
model.setObjective(
    sum(
        lambda_CES[t] * P_DSO_s[t] - lambda_CEP[t] * P_DSO_p[t] +
        lambda_EP[t] * sum(P_VPP_p[j, t] for j in J) -
        lambda_ES[t] * sum(P_VPP_s[j, t] for j in J)
        for t in T
    ),
    GRB.MAXIMIZE
)

model.optimize()

# Output
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")
print(f"Optimal Profit: {model.ObjVal}")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 18 columns and 51 nonzeros
Model fingerprint: 0xe14422d4
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 9e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [9e+00, 1e+02]
Presolve removed 24 rows and 18 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: -1.77636e-15 
No other solutions better than -1.77636e-15

Optimal solution found (tolerance 1.00e-04)
Best objective -1.776356839400e-15, best bound -1.776356839400e-15, gap 0.0000%
lambda_ES_1: 0.2
lambda_ES_2: 0

Fully Generalized UL

In [3]:
from gurobipy import Model, GRB

# Time periods and VPPs
T = [1, 2, 3]
J = [1, 2, 3]  # Multiple VPPs with same config

# Prices
lambda_CEP = {1: 0.4, 2: 0.6, 3: 1.4}  # Selling price to wholesale
lambda_CES = {1: 0.2, 2: 0, 3: 0}      # Buying price from wholesale
M = 100  # Big-M constant

# Lower-level solution placeholders (shared settings)
P_VPP_p = {(j, t): 3.0 for j in J for t in T}  # dummy values, normally from lower-level model
P_VPP_s = {(j, t): 0.0 for j in J for t in T}

# Create model
model = Model("DSO_Upper_Level_General")

# Variables
lambda_ES = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_ES_{t}") for t in T}
lambda_EP = {t: model.addVar(lb=lambda_CES[t], ub=lambda_CEP[t], name=f"lambda_EP_{t}") for t in T}
P_DSO = {t: model.addVar(name=f"P_DSO_{t}") for t in T}
P_DSO_p = {t: model.addVar(lb=0, name=f"P_DSO_p_{t}") for t in T}
P_DSO_s = {t: model.addVar(lb=0, name=f"P_DSO_s_{t}") for t in T}
z = {t: model.addVar(vtype=GRB.BINARY, name=f"z_{t}") for t in T}

# Constraints
for t in T:
    model.addConstr(P_DSO[t] == sum(P_VPP_p[j, t] - P_VPP_s[j, t] for j in J), name=f"Power_Balance_{t}")

    # Purchase Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO[t])
    model.addConstr(P_DSO[t] <= M * z[t])
    model.addConstr(-M * (1 - z[t]) <= P_DSO_p[t] - P_DSO[t])
    model.addConstr(P_DSO_p[t] <= M * z[t])

    # Sale Mode
    model.addConstr(-M * (1 - z[t]) <= P_DSO_s[t])
    model.addConstr(P_DSO_s[t] <= M * (1 - z[t]))
    model.addConstr(-M * z[t] <= P_DSO_s[t] + P_DSO[t])

# Objective
model.setObjective(
    sum(
        lambda_CES[t] * P_DSO_s[t] - lambda_CEP[t] * P_DSO_p[t] +
        lambda_EP[t] * sum(P_VPP_p[j, t] for j in J) -
        lambda_ES[t] * sum(P_VPP_s[j, t] for j in J)
        for t in T
    ),
    GRB.MAXIMIZE
)

model.optimize()

# Output
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")
print(f"Optimal Profit: {model.ObjVal}")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 18 columns and 51 nonzeros
Model fingerprint: 0xe14422d4
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 9e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [9e+00, 1e+02]
Presolve removed 24 rows and 18 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: -1.77636e-15 
No other solutions better than -1.77636e-15

Optimal solution found (tolerance 1.00e-04)
Best objective -1.776356839400e-15, best bound -1.776356839400e-15, gap 0.0000%
lambda_ES_1: 0.2
lambda_ES_2: 0